In [ ]:
def convolution_block(inputs, out_channels, name='conv'):
    conv = tf.layers.conv2d(inputs, out_channels, kernel_size=3, padding='SAME')
    conv = tf.contrib.layers.batch_norm(conv, updates_collections=None, decay=0.99, scale=True, center=True)
    conv = tf.nn.relu(conv)
    conv = tf.contrib.layers.max_pool2d(conv, 2)
    return conv

def encoder(support_set, h_dim, z_dim, reuse=False):
    net = convolution_block(support_set, h_dim, name='conv_1')
    net = convolution_block(net, h_dim, name='conv_2')
    net = convolution_block(net, h_dim, name='conv_3')
    net = convolution_block(net, z_dim, name='conv_4')
    net = tf.contrib.layers.flatten(net)
    return net

def euclidean_distance(a, b):
    N, D = tf.shape(a)[0], tf.shape(a)[1]
    M = tf.shape(b)[0]
    a = tf.tile(tf.expand_dims(a, axis=1), (1, M, 1))
    b = tf.tile(tf.expand_dims(b, axis=0), (N, 1, 1))
    return tf.reduce_mean(tf.square(a - b), axis=2)

In [ ]:

support_set = tf.placeholder(tf.float32, [None, None, img_height, img_width, channels])
query_set = tf.placeholder(tf.float32, [None, None, img_height, img_width, channels])
support_set_shape = tf.shape(support_set)
query_set_shape = tf.shape(query_set)
num_classes, num_support_points = support_set_shape[0], support_set_shape[1]
num_query_points = query_set_shape[1]
y = tf.placeholder(tf.int64, [None, None])
y_one_hot = tf.one_hot(y, depth=num_classes)
support_set_embeddings = encoder(tf.reshape(support_set, [num_classes * num_support_points, img_height, img_width, channels]), h_dim, z_dim)
embedding_dimension = tf.shape(support_set_embeddings)[-1]
class_prototype = tf.reduce_mean(tf.reshape(support_set_embeddings, [num_classes, num_support_points, embedding_dimension]), axis=1)
query_set_embeddings = encoder(tf.reshape(query_set, [num_classes * num_query_points, img_height, img_width, channels]), h_dim, z_dim, reuse=True)
distance = euclidean_distance(query_set_embeddings, class_prototype)
predicted_probability = tf.reshape(tf.nn.log_softmax(-distance), [num_classes, num_query_points, -1])
loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_one_hot, predicted_probability), axis=-1), [-1]))
accuracy = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(predicted_probability, axis=-1), y)))

train_op = tf.train.AdamOptimizer().minimize(loss)

sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)

for epoch in range(num_epochs):
    for episode in range(num_episodes):

        x_batch, _ = generator.batch('training')
        splits = np.array(np.array_split(x_batch, num_way))
        support, query = np.array_split(splits,[1], axis=1)

        labels = np.tile(np.arange(num_way)[:, np.newaxis], (1, num_query)).astype(np.uint8)
        _, loss_, accuracy_ = sess.run([train_op, loss, accuracy], feed_dict={support_set: support, query_set: query, y:labels})

        if (episode+1) % 10 == 0:
            print('Epoch {} : Episode {} : Loss: {}, Accuracy: {}'.format(epoch+1, episode+1, loss_, accuracy_))


            
# print('Testing...')

# avg_acc = 0.
# for epi in range(n_test_episodes):
#     epi_classes = np.random.permutation(x_test_classes)[:n_test_way]
#     support = np.zeros([n_test_way, n_test_shot, img_height, img_width], dtype=np.float32)
#     query = np.zeros([n_test_way, n_test_query, img_height, img_width], dtype=np.float32)
#     for i, epi_cls in enumerate(epi_classes):
#         selected = np.random.permutation(num_examples)[:n_test_shot + n_test_query]
#         support[i] = x_test[epi_cls, selected[:n_test_shot]]
#         query[i] = x_test[epi_cls, selected[n_test_shot:]]
#     support = np.expand_dims(support, axis=-1)
#     query = np.expand_dims(query, axis=-1)
#     labels = np.tile(np.arange(n_test_way)[:, np.newaxis], (1, n_test_query)).astype(np.uint8)
#     ls, ac = sess.run([loss, accuracy], feed_dict={support_set: support, query_set: query, y:labels})
#     avg_acc += ac
#     if (epi+1) % 50 == 0:
#         print('[test episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(epi+1, n_test_episodes, ls, ac))
# avg_acc /= n_test_episodes
# print('Average Test Accuracy: {:.5f}'.format(avg_acc))

In [3]:
from tensorflow.keras import backend as keras, regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D, Cropping2D, concatenate, Input, Reshape, AveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam


In [10]:
img_height, img_width, channels = 128,128,3
support_set = Input([None, img_height, img_width, channels])
query_set = Input([None, img_height, img_width, channels])

In [17]:
import tensorflow as tf

In [18]:
y_one_hot = tf.one_hot(y, depth=)

TypeError: one_hot() missing 2 required positional arguments: 'indices' and 'depth'